<a href="https://colab.research.google.com/github/viswavich/DLTBACS-PROJECT/blob/main/FINAL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries (if not already installed)
!pip install -q pandas numpy scikit-learn tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [3]:
# Load encoded and raw (lookup) datasets
df_encoded = pd.read_csv('/content/IT_Company_1000_Employees_Encoded.csv')
df_lookup = pd.read_csv('/content/Employee_Metadata_Lookup.csv')

# Load trained model (upload your model.h5 file)
model = load_model('/content/dlbac_light_model.h5')  # Upload your trained model file

In [4]:
# Encode all needed columns using LabelEncoder
label_cols = ['DEPT', 'ROLE', 'ACCESS_TIME', 'DEVICE_TRUST', 'RESOURCE_TYPE', 'SENSITIVITY_LEVEL']
encoders = {}
for col in label_cols:
    le = LabelEncoder()
    le.fit(df_lookup[col])
    encoders[col] = le

In [5]:
def get_access_decision(emp_id, resource_id):
    # Lookup record
    record = df_lookup[(df_lookup['EMP_ID'] == emp_id) & (df_lookup['RESOURCE_ID'] == resource_id)]

    if record.empty:
        return "🚫 Invalid EMP_ID or RESOURCE_ID", None

    # Display user role and dept
    role = record['ROLE'].values[0]
    dept = record['DEPT'].values[0]
    print(f"🔍 Found Record: Role = {role}, Department = {dept}\n")

    # Encode features
    features = [
        encoders['DEPT'].transform([record['DEPT'].values[0]])[0],
        encoders['ROLE'].transform([record['ROLE'].values[0]])[0],
        encoders['ACCESS_TIME'].transform([record['ACCESS_TIME'].values[0]])[0],
        encoders['DEVICE_TRUST'].transform([record['DEVICE_TRUST'].values[0]])[0],
        encoders['RESOURCE_TYPE'].transform([record['RESOURCE_TYPE'].values[0]])[0],
        encoders['SENSITIVITY_LEVEL'].transform([record['SENSITIVITY_LEVEL'].values[0]])[0],
    ]

    # Predict using model
    pred = model.predict(np.array(features).reshape(1, -1))[0][0]
    decision = "✅ Access Granted" if pred >= 0.5 else "❌ Access Denied"
    print(f"Prediction Score: {round(pred, 4)}")
    return decision, round(pred, 4)

In [7]:
# Example: You can change the EMP_ID and RESOURCE_ID based on your CSV
emp_id = 1013
resource_id = 'res974'

result, score = get_access_decision(emp_id, resource_id)
print("📢 Access Decision:", result)

🔍 Found Record: Role = Support, Department = R&D

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Prediction Score: 0.8877999782562256
📢 Access Decision: ✅ Access Granted


SAMPLE OUTPUT AND REASON

👤 EMP_ID: 1013
📂 RESOURCE_ID: res404

🔎 Role: Developer  
🏢 Department: IT

🧠 AI Model Decision:
✅ Access Granted  
Confidence: 0.89

Why? → Trusted device, role matches low sensitivity access